#### Data Preparation -- Universal Analytics 

***Note: Cleaned and transformed dataset using Pandas and preparing data for modeling/visualization.***

***Dataset: [Bitcoin Historical Data](https://www.kaggle.com/datasets/mczielinski/bitcoin-historical-data)*** 

In [1]:
import pandas as pd
import kagglehub
import os


In [2]:
# -----------------------------------------
# 1. DOWNLOAD DATASET FROM KAGGLE
# -----------------------------------------
print("Downloading dataset from Kaggle...")

path = kagglehub.dataset_download("mczielinski/bitcoin-historical-data")
print("Dataset downloaded to:", path)

# Locate the main CSV file
file_path = os.path.join(path, "btcusd_1-min_data.csv")


Dataset downloaded to: C:\Users\minha\.cache\kagglehub\datasets\mczielinski\bitcoin-historical-data\versions\416


In [3]:
# -----------------------------------------
# 2. LOAD DATA
# -----------------------------------------
print("Loading dataset...")

df = pd.read_csv(file_path)

df.head()


Loading dataset...


,Timestamp,Open,High,Low,Close,Volume
0,1.325412e+09,4.58,4.58,4.58,4.58,0.0
1,1.325412e+09,4.58,4.58,4.58,4.58,0.0
2,1.325412e+09,4.58,4.58,4.58,4.58,0.0
3,1.325412e+09,4.58,4.58,4.58,4.58,0.0
4,1.325412e+09,4.58,4.58,4.58,4.58,0.0


In [4]:
# -----------------------------------------
# 3. CLEAN & PREPROCESS
# -----------------------------------------

# Convert timestamp from Unix to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
df = df.rename(columns={'Timestamp': 'Date'})

# Set Date as index
df.set_index('Date', inplace=True)

# Inspect dataset
print("Initial rows:", len(df))
print(df.head())

# Remove duplicates if any
df = df[~df.index.duplicated(keep='first')]

# Fix missing values (forward-fill then back-fill)
df = df.ffill().bfill()

# Remove rows with zero BTC volume (invalid trading data)
df = df[df['Volume'] > 0]

print("Rows after cleaning:", len(df))


Initial rows: 7291837
                     Open  High   Low  Close  Volume
Date                                                
2012-01-01 10:01:00  4.58  4.58  4.58   4.58     0.0
2012-01-01 10:02:00  4.58  4.58  4.58   4.58     0.0
2012-01-01 10:03:00  4.58  4.58  4.58   4.58     0.0
2012-01-01 10:04:00  4.58  4.58  4.58   4.58     0.0
2012-01-01 10:05:00  4.58  4.58  4.58   4.58     0.0
Rows after cleaning: 5981378


In [6]:
# -----------------------------------------
# 4. RESAMPLE DATA 
# -----------------------------------------
# 1-minute data is very large and noisy — resample to daily or hourly

df_hourly = df.resample("1H").agg({
    "Open": "first",
    "High": "max",
    "Low": "min",
    "Close": "last",
    "Volume": "sum",
}).dropna()

df_daily = df.resample("1D").agg({
    "Open": "first",
    "High": "max",
    "Low": "min",
    "Close": "last",
    "Volume": "sum",
}).dropna()


C:\Users\minha\AppData\Local\Temp\ipykernel_21540\3805990564.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df.resample("1H").agg({


In [7]:
# -----------------------------------------
# 5. SAVE CLEANED DATA
# -----------------------------------------

output_hourly = "Datasets/clean_btcusd_hourly.csv"
output_daily = "Datasets/clean_btcusd_daily.csv"

df_hourly.to_csv(output_hourly)
df_daily.to_csv(output_daily)

print("Saved hourly cleaned dataset:", output_hourly)
print("Saved daily cleaned dataset:", output_daily)

print("Data cleaning complete.")


Saved hourly cleaned dataset: Datasets/clean_btcusd_hourly.csv
Saved daily cleaned dataset: Datasets/clean_btcusd_daily.csv
Data cleaning complete.
